这里说频率是指一个机场航班出现的频率

## 加载数据

数据已经预处理过了

In [1]:
import pandas as pd


# 加载数据
pd.set_option('display.max_columns', None)  # 显示所有列

data = pd.read_csv('../../../../data-hh/2025/2025-5/result/pre_2023-2025_with_comp_test.csv', dtype={'aircraft': str})

# 查看前几行数据，确保加载成功
print(data.head())

print(data.info())

   flt_no    cap aircraft  legs  leg_no  duration  pax       a       b  \
0  3c6705   94.0      190     1       1      1.33   45  a9f275  24cc4b   
1  284078  162.0      322     1       1      2.10  143  e3d141  d618e3   
2  484808  169.0      738     1       1      4.52  154  9550bc  2e1ea1   
3  0f40c9  167.0      320     1       1      4.28  167  9550bc  2e1ea1   
4  ce2225  166.0      320     1       1      1.33  181  9550bc  24cc4b   

        c   unit_price  competitor_price  year  month  day  weekday  hour  \
0  7f9c2b   481.555556       -119.744444  2024      7    1        0    12   
1  7f9c2b   902.167832          0.000000  2024      7    1        0    10   
2  7f9c2b  1540.357143        283.690476  2024      7    1        0    14   
3  7f9c2b  1345.449102        960.619556  2024      7    1        0    21   
4  7f9c2b   593.480663        -54.049640  2024      7    1        0    23   

   minute from   to  
0      30  a9f  c4b  
1      30  e3d  8e3  
2      15  955  ea1  
3   

In [2]:
# # 随机抽取5万条数据
# data = data.sample(n=20000, random_state=42)

# 查看抽样后的数据信息
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282641 entries, 0 to 282640
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   flt_no            282641 non-null  object 
 1   cap               282641 non-null  float64
 2   aircraft          282641 non-null  object 
 3   legs              282641 non-null  int64  
 4   leg_no            282641 non-null  int64  
 5   duration          282609 non-null  float64
 6   pax               282641 non-null  int64  
 7   a                 282641 non-null  object 
 8   b                 282641 non-null  object 
 9   c                 282641 non-null  object 
 10  unit_price        282641 non-null  float64
 11  competitor_price  282641 non-null  float64
 12  year              282641 non-null  int64  
 13  month             282641 non-null  int64  
 14  day               282641 non-null  int64  
 15  weekday           282641 non-null  int64  
 16  hour              28

In [3]:
# 检查标准化后的统计信息
print("\n标准化后的统计信息：")
print(data['pax'].describe())


标准化后的统计信息：
count    282641.000000
mean        131.741322
std          51.102982
min          20.000000
25%          98.000000
50%         140.000000
75%         164.000000
max         301.000000
Name: pax, dtype: float64


## 编码分类变量

### 新增城市标签

In [4]:
import json
# 加载字典
with open('../../../../data-hh/2025/2025-5/result/encoder/city_labels_航班频率加权图标签.json', 'r') as file:
    city_labels_loaded = json.load(file)

print("加载的字典：", city_labels_loaded)

加载的字典： {'000': 1, '13f': 0, '8e3': 0, 'a18': 0, 'eda': 2, '01a': 1, 'b5a': 0, 'ea1': 0, '03f': 2, '1be': 0, '3bc': 0, '578': 0, '5df': 0, '5fb': 0, '760': 0, '792': 0, '7ff': 0, '90a': 2, '974': 0, 'bbb': 0, 'cb5': 0, 'ffe': 0, '059': 2, '022': 0, '077': 0, '0d7': 0, '0dd': 0, '1a3': 0, '1e9': 0, '1ed': 0, '25c': 0, '311': 0, '354': 0, '451': 0, '510': 0, '5d7': 0, '63a': 0, '666': 0, '6e3': 0, '7f1': 0, '812': 0, '8a1': 0, '9d4': 2, '9e9': 0, 'b3a': 0, 'bff': 0, 'c24': 0, 'c3a': 0, 'c4c': 0, 'c98': 0, 'c9f': 0, 'cb7': 0, 'd7c': 0, 'e76': 0, 'e9c': 0, 'efb': 0, '05c': 2, '141': 0, '157': 0, '709': 0, '8ac': 0, '97f': 0, '988': 0, 'a2f': 0, 'b09': 0, 'c14': 0, 'c4b': 0, 'e5b': 0, 'ec2': 0, '074': 1, '4d0': 0, '695': 0, 'c2c': 0, 'fa9': 0, '08c': 1, '08f': 2, '3c5': 0, '5e8': 0, 'a72': 0, 'd54': 0, 'ff1': 0, '0ac': 1, '0c8': 1, '0ca': 1, 'e09': 0, '0d1': 1, '13e': 0, '1ee': 0, '43e': 0, '0db': 2, '919': 0, 'd08': 0, '0ff': 2, '11b': 1, '123': 2, '5de': 0, '8eb': 0, '124': 1, '130': 2, '1

In [5]:
# 使用 map 对 'a', 'b', 'c', 'from', 'to' 列进行标签化，新增对应的标签列
data['a_label'] = data['a'].map(city_labels_loaded)
data['b_label'] = data['b'].map(city_labels_loaded)
data['c_label'] = data['c'].map(city_labels_loaded)
data['from_label'] = data['from'].map(city_labels_loaded)
data['to_label'] = data['to'].map(city_labels_loaded)

### 新增城市二维嵌入

In [6]:
import json

# 加载字典
with open('../../../../data-hh/2025/2025-5/result/encoder/城市嵌入编码_航班频率加权图.json', 'r') as file:
    city_embeddings = json.load(file)

print("加载的字典：", city_embeddings)


加载的字典： {'000': [-1.6396723985671997, 0.8463003039360046], '13f': [0.6835631728172302, 0.28853702545166016], '8e3': [0.605768084526062, 0.43108901381492615], 'a18': [0.1772567182779312, 0.35664066672325134], 'eda': [-0.18683721125125885, 0.786605715751648], '01a': [-1.6955363750457764, 0.015374363400042057], 'b5a': [0.6262921690940857, 0.6344022154808044], 'ea1': [0.2808123230934143, -0.011203848756849766], '03f': [-0.929985761642456, 1.23349928855896], '1be': [0.8312583565711975, 0.8889409899711609], '3bc': [1.4587459564208984, 1.2139819860458374], '578': [0.9716920256614685, 0.5555941462516785], '5df': [0.8122007846832275, 1.0064148902893066], '5fb': [0.7514726519584656, 0.360472708940506], '760': [1.288560390472412, 0.6948064565658569], '792': [0.14092816412448883, 0.706443727016449], '7ff': [1.017447829246521, 0.27192866802215576], '90a': [0.07465023547410965, 0.3252807855606079], '974': [1.2340888977050781, 0.8395470380783081], 'bbb': [0.4277801811695099, 0.7776517868041992], 'cb5'

In [7]:
import pandas as pd
import numpy as np

# 将 city_embeddings 转换为 DataFrame
embedding_df = pd.DataFrame.from_dict(city_embeddings, orient='index', columns=['embedding_1', 'embedding_2'])
embedding_df.index.name = 'city'

# 用 'a', 'b', 'c', 'from', 'to' 字段与 embedding_df 合并
data = data.merge(embedding_df, left_on='a', right_index=True, how='left')
data.rename(columns={'embedding_1': 'a_embedding_1', 'embedding_2': 'a_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='b', right_index=True, how='left')
data.rename(columns={'embedding_1': 'b_embedding_1', 'embedding_2': 'b_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='c', right_index=True, how='left')
data.rename(columns={'embedding_1': 'c_embedding_1', 'embedding_2': 'c_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='from', right_index=True, how='left')
data.rename(columns={'embedding_1': 'from_embedding_1', 'embedding_2': 'from_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='to', right_index=True, how='left')
data.rename(columns={'embedding_1': 'to_embedding_1', 'embedding_2': 'to_embedding_2'}, inplace=True)

# 查看添加的新列
print(data[['a_embedding_1', 'a_embedding_2', 'b_embedding_1', 'b_embedding_2', 'c_embedding_1', 'c_embedding_2', 'from_embedding_1', 'from_embedding_2', 'to_embedding_1', 'to_embedding_2']])

        a_embedding_1  a_embedding_2  b_embedding_1  b_embedding_2  \
0                 NaN            NaN            NaN            NaN   
1                 NaN            NaN            NaN            NaN   
2                 NaN            NaN            NaN            NaN   
3                 NaN            NaN            NaN            NaN   
4                 NaN            NaN            NaN            NaN   
...               ...            ...            ...            ...   
282636            NaN            NaN            NaN            NaN   
282637            NaN            NaN            NaN            NaN   
282638            NaN            NaN            NaN            NaN   
282639            NaN            NaN            NaN            NaN   
282640            NaN            NaN            NaN            NaN   

        c_embedding_1  c_embedding_2  from_embedding_1  from_embedding_2  \
0                 NaN            NaN         -1.642773         -1.587628   
1      

### 统计不同城市的频率

### 对'a', 'b', 'c', 'from', 'to'进行频率编码

In [8]:
# 加载city_map
with open('../../../../data-hh/2025/2025-5/result/encoder/city_map_频率编码.json', 'r') as f:
    city_map = json.load(f)

# 使用 city_map 替换指定列的值
columns_to_replace = ['a', 'b', 'c', 'from', 'to']

# 遍历指定列并直接用 map 映射
for col in columns_to_replace:
    data[col] = data[col].map(city_map)


print(data)

        flt_no    cap aircraft  legs  leg_no  duration  pax   a   b   c  \
0       3c6705   94.0      190     1       1      1.33   45 NaN NaN NaN   
1       284078  162.0      322     1       1      2.10  143 NaN NaN NaN   
2       484808  169.0      738     1       1      4.52  154 NaN NaN NaN   
3       0f40c9  167.0      320     1       1      4.28  167 NaN NaN NaN   
4       ce2225  166.0      320     1       1      1.33  181 NaN NaN NaN   
...        ...    ...      ...   ...     ...       ...  ...  ..  ..  ..   
282636  2befc2  161.0      738     1       1      2.57  155 NaN NaN NaN   
282637  4cf718  167.0      32C     3       3      3.35   29 NaN NaN NaN   
282638  5951a4  173.0      73Z     1       1      2.57  166 NaN NaN NaN   
282639  408f86   95.0      190     3       2      1.70   85 NaN NaN NaN   
282640  b60fa0   95.0      190     3       2      2.25   67 NaN NaN NaN   

         unit_price  competitor_price  year  month  day  weekday  hour  \
0        481.555556      

### 对'flt_no', 'bd_type', 'aircraft'进行标签编码

In [9]:
data.head(5)

,flt_no,cap,aircraft,legs,leg_no,duration,pax,a,b,c,unit_price,competitor_price,year,month,day,weekday,hour,minute,from,to,a_label,b_label,c_label,from_label,to_label,a_embedding_1,a_embedding_2,b_embedding_1,b_embedding_2,c_embedding_1,c_embedding_2,from_embedding_1,from_embedding_2,to_embedding_1,to_embedding_2
0,3c6705,94.0,190,1,1,1.33,45,NaN,NaN,NaN,481.555556,-119.744444,2024,7,1,0,12,30,1382,45311,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,-1.642773,-1.587628,0.162349,-1.332757
1,284078,162.0,322,1,1,2.10,143,NaN,NaN,NaN,902.167832,0.000000,2024,7,1,0,10,30,544,45328,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,-1.310817,0.426924,0.605768,0.431089
2,484808,169.0,738,1,1,4.52,154,NaN,NaN,NaN,1540.357143,283.690476,2024,7,1,0,14,15,3415,87446,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,-1.786745,-1.186316,0.280812,-0.011204
3,0f40c9,167.0,320,1,1,4.28,167,NaN,NaN,NaN,1345.449102,960.619556,2024,7,1,0,21,30,3415,87446,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,-1.786745,-1.186316,0.280812,-0.011204
4,ce2225,166.0,320,1,1,1.33,181,NaN,NaN,NaN,593.480663,-54.049640,2024,7,1,0,23,15,3415,45311,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,-1.786745,-1.186316,0.162349,-1.332757


In [10]:
import joblib
from sklearn.preprocessing import LabelEncoder
import os

# 定义需要编码的分类特征
# categorical_columns = ['flt_no', 'bd_type', 'aircraft']
categorical_columns = ['flt_no',  'aircraft']

# 从保存的文件中加载编码器并应用到data
for col in categorical_columns:
    # 加载编码器
    encoder_path = os.path.join('../../../../data-hh/2025/2025-5/result/encoder/', f"{col}_encoder_all.pkl")
    le = joblib.load(encoder_path)
    
    try:
        # 对data进行转换
        data[col] = le.transform(data[col])
        print(f"{col}列编码完成")
    except ValueError as e:
        # 如果遇到新的类别，打印错误信息
        print(f"{col}列编码出错: {str(e)}")
        # 找出新的类别
        new_categories = set(data[col]) - set(le.classes_)
        print(f"{col}列中的新类别: {new_categories}")

# 查看编码后的结果
print("\n编码后的前几行数据：")
print(data[categorical_columns].head())

flt_no列编码完成
aircraft列编码完成

编码后的前几行数据：
   flt_no  aircraft
0    1097         0
1     719        11
2    1301        48
3     277         9
4    3585         9


## 特征和目标分离
我们要预测的是pax字段，其他字段作为特征。

In [11]:
# 特征列
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft',  'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','holiday', 'hour', 'minute', 'second', 'from', 'to','unit_price']]
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price']]
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label']]

# 有abc，有标签，有嵌入
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]
X = data[['flt_no', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'from', 'to','unit_price','competitor_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]
# 删除了abc，但有标签，有嵌入
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]

# 目标列
y = data['pax']

In [12]:
test_x = X.head(1)
test_x

,flt_no,cap,aircraft,legs,leg_no,duration,a,b,c,year,month,day,weekday,hour,minute,from,to,unit_price,competitor_price,a_label,b_label,c_label,from_label,to_label,a_embedding_1,a_embedding_2,b_embedding_1,b_embedding_2,c_embedding_1,c_embedding_2,from_embedding_1,from_embedding_2,to_embedding_1,to_embedding_2
0,1097,94.0,0,1,1,1.33,NaN,NaN,NaN,2024,7,1,0,12,30,1382,45311,481.555556,-119.744444,NaN,NaN,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,-1.642773,-1.587628,0.162349,-1.332757


In [13]:
y.head(5)

0     45
1    143
2    154
3    167
4    181
Name: pax, dtype: int64

### 对x进行标准化

In [14]:
from sklearn.preprocessing import StandardScaler
import joblib

# 定义需要标准化的特征列（与训练时相同的特征）
features = ['flt_no', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 
           'a', 'b', 'c', 'year', 'month', 'day', 'weekday', 'hour', 'minute', 
           'from', 'to', 'unit_price','competitor_price', 'a_label', 'b_label', 'c_label', 'from_label', 'to_label',
           'a_embedding_1', 'a_embedding_2', 'b_embedding_1', 'b_embedding_2', 
           'c_embedding_1', 'c_embedding_2', 'from_embedding_1', 'from_embedding_2', 
           'to_embedding_1', 'to_embedding_2']

# 加载保存的标准化器
scaler = joblib.load('../../../../data-hh/2025/2025-5/result/encoder/standard_scaler_x.pkl')

# 对数据进行标准化
data_scaled = scaler.transform(data[features])

# 将标准化后的数据转回DataFrame格式
data_scaled = pd.DataFrame(data_scaled, columns=features, index=data.index)

# 将标准化后的数据替换回原始数据框中
for col in features:
    data[col] = data_scaled[col]

# 查看标准化后的结果
print("标准化后的数据预览（前1行）：")
pd.set_option('display.max_columns', None)  # 显示所有列
print(data[features].head(1))

# 检查标准化后的统计信息
print("\n标准化后的统计信息：")
print(data[features].describe())

标准化后的数据预览（前1行）：
     flt_no       cap  aircraft      legs    leg_no  duration   a   b   c  \
0 -0.849957 -1.971191 -1.480375 -0.850028 -0.557392 -1.208322 NaN NaN NaN   

       year    month       day   weekday      hour    minute      from  \
0  0.334892  0.46562 -1.681813 -1.513001 -0.401862  0.204208 -1.345293   

         to  unit_price  competitor_price  a_label  b_label  c_label  \
0  0.207053   -0.533106         -0.542197      NaN      NaN      NaN   

   from_label  to_label  a_embedding_1  a_embedding_2  b_embedding_1  \
0   -3.667879 -0.305874            NaN            NaN            NaN   

   b_embedding_2  c_embedding_1  c_embedding_2  from_embedding_1  \
0            NaN            NaN            NaN         -2.402695   

   from_embedding_2  to_embedding_1  to_embedding_2  
0         -3.013874       -1.057717       -2.891073  

标准化后的统计信息：
              flt_no            cap       aircraft           legs  \
count  282641.000000  282641.000000  282641.000000  282641.00000

### 对y进行标准化

In [15]:
# 加载y的标准化器
scaler_y = joblib.load('../../../../data-hh/2025/2025-5/result/encoder/standard_scaler_y.pkl')

# 对目标变量进行标准化
y = data['pax']  # 假设目标列名为'pax'
# y_scaled = scaler_y.transform(y.values.reshape(-1, 1))  # 将y转换为2D数组进行标准化

# # 将标准化后的数据转回DataFrame格式
# y_scaled = pd.DataFrame(y_scaled, columns=['pax_scaled'], index=data.index)

# # 将标准化后的值添加到原始数据框中
# data['pax'] = y_scaled['pax_scaled']

## XGBoost模型预测

In [16]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# 加载模型
model = xgb.Booster()
model.load_model("../../../../data-hh/2025/2025-5/result/model/归一化_xgboost_model_1000.json")

# 转换为DMatrix格式
dmatrix = xgb.DMatrix(data_scaled)

# 进行预测
scaled_predictions = model.predict(dmatrix)

# 将标准化的预测值转换回原始尺度
predictions = scaler_y.inverse_transform(scaled_predictions.reshape(-1, 1))

In [17]:
predictions

array([[ 62.571037],
       [148.04253 ],
       [173.17781 ],
       ...,
       [163.37856 ],
       [ 76.753174],
       [ 70.056625]], dtype=float32)

In [18]:
# 正确的写法
test_results = list(zip(predictions[:100], y[:100]))  # 真实值和预测值

print("\n20条测试结果（真实值 vs 预测值）:")
for i, (true_value, pred_value) in enumerate(test_results[:40]):
    # 如果是多维数组，使用 .item() 转换为标量
    true_value = true_value.item() if isinstance(true_value, np.ndarray) else true_value
    pred_value = pred_value.item() if isinstance(pred_value, np.ndarray) else pred_value
    print(f"第{i+1}条: 真实值={true_value}, 预测值={pred_value:.2f}")



20条测试结果（真实值 vs 预测值）:
第1条: 真实值=62.57103729248047, 预测值=45.00
第2条: 真实值=148.0425262451172, 预测值=143.00
第3条: 真实值=173.1778106689453, 预测值=154.00
第4条: 真实值=172.81842041015625, 预测值=167.00
第5条: 真实值=143.0325927734375, 预测值=181.00
第6条: 真实值=108.48690795898438, 预测值=138.00
第7条: 真实值=92.0132064819336, 预测值=137.00
第8条: 真实值=45.359622955322266, 预测值=50.00
第9条: 真实值=106.51458740234375, 预测值=136.00
第10条: 真实值=78.61088562011719, 预测值=99.00
第11条: 真实值=59.72386932373047, 预测值=70.00
第12条: 真实值=89.67015075683594, 预测值=62.00
第13条: 真实值=62.59638214111328, 预测值=57.00
第14条: 真实值=54.767337799072266, 预测值=67.00
第15条: 真实值=162.73422241210938, 预测值=167.00
第16条: 真实值=55.15969467163086, 预测值=30.00
第17条: 真实值=93.76014709472656, 预测值=102.00
第18条: 真实值=116.34385681152344, 预测值=163.00
第19条: 真实值=81.25579071044922, 预测值=42.00
第20条: 真实值=82.62184143066406, 预测值=119.00
第21条: 真实值=50.40524673461914, 预测值=24.00
第22条: 真实值=159.27244567871094, 预测值=161.00
第23条: 真实值=163.6890869140625, 预测值=162.00
第24条: 真实值=173.0194091796875, 预测值=170.00
第25条: 真实值=174.444091796875, 预测

## 测试指标

In [19]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def calculate_smape(y_true, y_pred):
    """
    计算 Symmetric Mean Absolute Percentage Error (SMAPE)
    """
    y_true, y_pred = np.array(y_true).ravel(), np.array(y_pred).ravel()
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_pred - y_true)
    
    # 避免除以零，将分母中为零的项替换为一个小值
    denominator = np.where(denominator == 0, 1e-8, denominator)
    
    smape = 100 * np.mean(diff / denominator)
    return smape


def calculate_mape(y_true, y_pred):
    """
    计算 Mean Absolute Percentage Error (MAPE)
    """
    y_true, y_pred = np.array(y_true).ravel(), np.array(y_pred).ravel()
    
    # 避免除以零，将 y_true 中的零值替换为一个小值
    y_true = np.where(y_true == 0, 1e-8, y_true)
    
    mape = 100 * np.mean(np.abs((y_true - y_pred) / y_true))
    return mape


# 假设 y_test 和 y_pred 已经是标准化反归一化后的数据
# 将其转换为一维数组以确保形状一致
y_test = np.array(y).ravel()
y_pred = np.array(predictions).ravel()

# 评估指标
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = calculate_mape(y_test, y_pred)
smape = calculate_smape(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 打印结果
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.4f}%')
print(f'Symmetric Mean Absolute Percentage Error (SMAPE): {smape:.4f}%')
print(f'R-squared (R²): {r2:.4f}')


Mean Squared Error (MSE): 424.8919
Root Mean Squared Error (RMSE): 20.6129
Mean Absolute Error (MAE): 15.2385
Mean Absolute Percentage Error (MAPE): 16.3113%
Symmetric Mean Absolute Percentage Error (SMAPE): 14.5927%
R-squared (R²): 0.8373


似乎对于较小值预测存在误差

In [20]:
def calculate_smape(y_true, y_pred):
    """
    计算 Symmetric Mean Absolute Percentage Error (SMAPE)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    smape = 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
    return smape

def calculate_mape(y_true, y_pred):
    """
    计算 Mean Absolute Percentage Error (MAPE)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = 100 * np.mean(np.abs((y_true - y_pred) / y_true))
    return mape

In [21]:
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# import numpy as np

# # 评估模型
# mse = mean_squared_error(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# mape = calculate_mape(y_test, y_pred)
# smape = calculate_smape(y_test, y_pred)

# # 打印结果
# print(f'Mean Squared Error (MSE): {mse:.4f}')
# print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
# print(f'Mean Absolute Error (MAE): {mae:.4f}')
# print(f'Mean Absolute Percentage Error (MAPE): {mape:.4f}%')
# print(f'Symmetric Mean Absolute Percentage Error (SMAPE): {smape:.4f}%')

## 保存模型

## 超参数设置

## 不同特征重要程度测试

In [22]:
# import xgboost as xgb
# import matplotlib.pyplot as plt

# # 假设 model 是训练好的 XGBoost 模型
# xgb.plot_importance(model, importance_type='weight', title="Feature Importance (Weight)", height=0.5)
# plt.show()

# xgb.plot_importance(model, importance_type='gain', title="Feature Importance (Gain)", height=0.5)
# plt.show()

# xgb.plot_importance(model, importance_type='cover', title="Feature Importance (Cover)", height=0.5)
# plt.show()